#DATASCI W261: Machine Learning at Scale 

#This notebook provides a poor man Hadoop through command-line and python. Please insert the python code by yourself.

#Map

In [25]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re, os
count = 0
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[2]
findword = sys.argv[1]
with open (filename, "r") as myfile:
    for line in myfile:
        for findword in line:
            count = count + 1
print (str(count))

Overwriting mapper.py


In [26]:
!chmod a+x mapper.py

#Reduce

In [27]:
%%writefile reducer.py
#!/usr/bin/python
import sys
sum = 0
for line in sys.stdin:
    sum = sum + int(line.strip())
print (str(sum))

Overwriting reducer.py


In [28]:
!chmod a+x reducer.py

# Write script to file

In [22]:
%%writefile pGrepCount.sh
ORIGINAL_FILE=$1
FIND_WORD=$2
BLOCK_SIZE=$3
CHUNK_FILE_PREFIX=$ORIGINAL_FILE.split
SORTED_CHUNK_FILES=$CHUNK_FILE_PREFIX*.sorted
usage()
{
    echo Parallel grep
    echo usage: pGrepCount filename word chuncksize
    echo greps file file1 in $ORIGINAL_FILE and counts the number of lines
    echo Note: file1 will be split in chunks up to $ BLOCK_SIZE chunks each
    echo $FIND_WORD each chunk will be grepCounted in parallel
}
#Splitting $ORIGINAL_FILE INTO CHUNKS
split -b $BLOCK_SIZE $ORIGINAL_FILE $CHUNK_FILE_PREFIX
#DISTRIBUTE
for file in $CHUNK_FILE_PREFIX*
do
    #grep -i $FIND_WORD $file|wc -l >$file.intermediateCount &
    ./mapper.py $FIND_WORD $file >$file.intermediateCount &
done
wait
#MERGEING INTERMEDIATE COUNT CAN TAKE THE FIRST COLUMN AND TOTOL...
#numOfInstances=$(cat *.intermediateCount | cut -f 1 | paste -sd+ - |bc)
numOfInstances=$(cat *.intermediateCount | ./reducer.py)
echo "found [$numOfInstances] [$FIND_WORD] in the file [$ORIGINAL_FILE]"

Overwriting pGrepCount.sh


#Run the file

In [23]:
!chmod a+x pGrepCount.sh

Usage: usage: pGrepCount filename word chuncksize

In [24]:
!./pGrepCount.sh License.txt COPYRIGHT 4k

found [<_io.TextIOWrapper name='License.txt.splitaa' mode='r' encoding='UTF-8'>

0

<_io.TextIOWrapper name='License.txt.splitaa.intermediateCount' mode='r' encoding='UTF-8'>

0

<_io.TextIOWrapper name='License.txt.splitaa.intermediateCount.intermediateCount' mode='r' encoding='UTF-8'>

0

<_io.TextIOWrapper name='License.txt.splitab' mode='r' encoding='UTF-8'>

0

<_io.TextIOWrapper name='License.txt.splitab.intermediateCount' mode='r' encoding='UTF-8'>

0

<_io.TextIOWrapper name='License.txt.splitab.intermediateCount.intermediateCount' mode='r' encoding='UTF-8'>

0

<_io.TextIOWrapper name='License.txt.splitac' mode='r' encoding='UTF-8'>

0

<_io.TextIOWrapper name='License.txt.splitac.intermediateCount' mode='r' encoding='UTF-8'>

0

<_io.TextIOWrapper name='License.txt.splitac.intermediateCount.intermediateCount' mode='r' encoding='UTF-8'>

0

<_io.TextIOWrapper name='License.txt.splitad' mode='r' encoding='UTF-8'>

0

<_io.TextIOWrapper name='License.txt.splitad.intermediateCoun